# TotalEnergies
# Qualification Hackathon

# Problem Statement

# Contents

# install libraries

In [ ]:
%%capture
!pip install -q -U watermark
!pip install -q -U pandas-profiling
!pip install -q -U pandas
!pip install -q -U numpy
!pip install -q -U matplotlib
!pip install -q -U seaborn
!pip install -q -U scikit-learn
!pip install -q -U xgboost
!pip install -q -U lightgbm
!pip install -q -U missingno
!pip intall -q -U tqdm
!pip install -q  -U google.colab

# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
from tqdm import tqdm
from google.colab import drive


# Load data

In [ ]:
drive.mount("/content/drive")

In [ ]:
test_url = "https://api.zindi.africa/v1/competitions/umojahack-africa-2023-beginner-challenge/files/Test.csv"
train_url= "https://api.zindi.africa/v1/competitions/umojahack-africa-2023-beginner-challenge/files/Train.csv"
submission_url = "https://api.zindi.africa/v1/competitions/umojahack-africa-2023-beginner-challenge/files/SampleSubmission.csv"

# auth token
token = {'auth_token' :'zus.v1.18SARW.xDYQ6sX7KYvcWFrvwWWtzvBBPoJDKa'}

In [ ]:
# load data
# zindi data loading function
def zindi_data_downloader(url, token, file_name):
    # Get the competition data
    competition_data = requests.post(url = url, data= token, stream=True)

    # Progress bar monitor download
    pbar = tqdm(desc=file_name, total=int(competition_data.headers.get('content-length', 0)), unit='B', unit_scale=True, unit_divisor=512)
    # Create and Write the data to colab drive in chunks
    handle = open(file_name, "wb")
    for chunk in competition_data.iter_content(chunk_size=512): # Download the data in chunks
        if chunk: # filter out keep-alive new chunks
            handle.write(chunk)
        pbar.update(len(chunk))
    handle.close()
    pbar.close()

In [ ]:
# download the hackthon files in drive
# filenames
test = "Test.csv"
train = "Train.csv"
submission = "SampleSubmission.csv"

# download the data
zindi_data_downloader(test_url,token,test)
zindi_data_downloader(train_url,token,train)
zindi_data_downloader(submission_url,token,submission)

In [ ]:
DATA_PATH = '/content/'
# Load files
train = pd.read_csv(os.path.join(DATA_PATH, 'Train.csv'))
test = pd.read_csv(os.path.join(DATA_PATH, 'Test.csv'))
samplesubmission = pd.read_csv(os.path.join(DATA_PATH, 'SampleSubmission.csv'))

# Preview train dataset
train.head()

# Exploratory Data Analysis

# Data Description

In [ ]:
# check the shape of the data
print("Train shape: ", train.shape)
print("Test shape: ", test.shape)
print("SampleSubmission shape: ", samplesubmission.shape)

# Data Cleaning

# Feature Engineering


# Modeling


# Model Evaluation


# Model Interpretation


# Model Deployment



# Conclusion


# submission

In [ ]:
# # Create file
counter = 0;
sub_file = pd.DataFrame({'ID': test['ID'], 'target': test_preds})
sub_file.to_csv(f'submissions/submission{counter}.csv', index = False) # Download subfile and submit to zindi for scoring-generate submission file
counter += 1


# References
